In [ ]:
import openmeteo_requests
import matplotlib.pyplot as plt
import requests_cache
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from retry_requests import retry
from matplotlib.dates import DateFormatter, DayLocator

In [ ]:
def call_open_meteo_min(selected_station, start, end):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
    retry_strategy = Retry(total=5, backoff_factor=0.2)
    adapter = HTTPAdapter(max_retries=retry_strategy)
    cache_session.mount('http://', adapter)
    cache_session.mount('https://', adapter)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    # Initialize an empty list to store daily data
    daily_data = []

    # Define the start and end dates for the loop
    start_date = pd.Timestamp(start)
    end_date = pd.Timestamp(end)

    # Iterate through each day of December
    while start_date <= end_date:
        # Define the parameters for the API call
        params = {
            "latitude": selected_station["lat"],
            "longitude": selected_station["lon"],
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": start_date.strftime("%Y-%m-%d"),  # Use the same start and end date for daily data
            "daily": "wind_speed_10m_min",
            "timezone": "Europe/Berlin",
        }

        # Make the API call
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)

        for response in responses:
            # Process daily data. The order of variables needs to be the same as requested.
            daily = response.Daily()
            daily_wind_speed_10m_min = daily.Variables(0).ValuesAsNumpy()

            # Calculate the average wind speed for the day
            min_wind_speed = daily_wind_speed_10m_min.min()

            # Append the average wind speed for the day to daily_data
            daily_data.append({
                "date": start_date.strftime('%Y-%m-%d'),  # Format date to include year, month, and day
                "wind_speed_10m_min": min_wind_speed,  # Use the average wind speed
            })

        # Move to the next day
        start_date += pd.Timedelta(days=1)

    # Create a DataFrame from daily data
    daily_dataframe = pd.DataFrame(daily_data)
    
    # Calculate daily averages
    daily_min = daily_dataframe.groupby('date').min()
    return daily_min

In [ ]:
def call_open_meteo_averages(selected_station, start, end):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
    retry_strategy = Retry(total=5, backoff_factor=0.2)
    adapter = HTTPAdapter(max_retries=retry_strategy)
    cache_session.mount('http://', adapter)
    cache_session.mount('https://', adapter)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    # Initialize an empty list to store daily data
    daily_data = []

    # Define the start and end dates for the loop
    start_date = pd.Timestamp(start)
    end_date = pd.Timestamp(end)

    # Iterate through each day of December
    while start_date <= end_date:
        # Define the parameters for the API call
        params = {
            "latitude": selected_station["lat"],
            "longitude": selected_station["lon"],
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": start_date.strftime("%Y-%m-%d"),  # Use the same start and end date for daily data
            "daily": "wind_speed_10m_mean",
            "timezone": "Europe/Berlin",
        }

        # Make the API call
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)

        for response in responses:
            # Process daily data. The order of variables needs to be the same as requested.
            daily = response.Daily()
            daily_wind_speed_10m_mean = daily.Variables(0).ValuesAsNumpy()

            # Calculate the average wind speed for the day
            avg_wind_speed = daily_wind_speed_10m_mean.mean()

            # Append the average wind speed for the day to daily_data
            daily_data.append({
                "date": start_date.strftime('%Y-%m-%d'),  # Format date to include year, month, and day
                "wind_speed_10m_mean": avg_wind_speed,  # Use the average wind speed
            })

        # Move to the next day
        start_date += pd.Timedelta(days=1)

    # Create a DataFrame from daily data
    daily_dataframe = pd.DataFrame(daily_data)
    
    # Calculate daily averages
    daily_avg = daily_dataframe.groupby('date').mean()
    return daily_avg

In [ ]:
def plot_everything(Wind_min, Wind_mean,Wind_max,temp_mean):
    data = pd.concat([Wind_min,Wind_max, Wind_mean], axis=1)
    data.columns = ['Wind Minimum','Wind Maximum', 'Wind Mittelwert']
    
    # Plot
    ax = data.plot(kind = 'bar', figsize=(100, 15))
    # Umbenennen der Spalte mit dem richtigen Label
    temp_mean.rename(columns={'temperature_2m_mean': 'Temperatur Mittelwert'}, inplace=True)

    # Hinzufügen der Linie für die Temperaturdaten
    temp_mean.plot(ax=ax, color='red', marker='', linewidth=2, label='Temperatur Mittelwert')
    
    # Beschriftungen und Anpassungen
    plt.xlabel('Datum')
    plt.ylabel('wind in m/s und Temp in °C')
    plt.title('Vergleich der Wetterdaten')
    plt.xticks(rotation=45)
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

In [ ]:
def plot_windmax(Wind_max):
    data = pd.concat([Wind_max], axis=1)
    data.columns = ['Wind Maximum']
    
    # Plot
    ax = data.plot(kind = 'line', figsize=(30, 6), linewidth=5, color='red')    
    # Beschriftungen und Anpassungen
    plt.xlabel('Datum')
    plt.ylabel('wind in m/s')
    plt.title('Wind Maximum beim Sturm Lothar')
    plt.xticks(rotation=45)
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

In [ ]:
def plot_windmax_compare(Wind_lothar, Wind_other, Vergleichsjahr, Ortschaft):
    # Synchronisieren der Indexe der beiden Datenrahmen
    Wind_other = Wind_other.set_index(Wind_lothar.index)
    other_year = Wind_other.index[0][:4]  # Extrahiere die ersten vier Zeichen, die das Jahr darstellen
    data = pd.concat([Wind_lothar, Wind_other], axis=1)
    data.columns = ['Orkan Lothar', f'Vergleichsjahr ({Vergleichsjahr})']
        
    # Plot
    ax = data.plot(kind='line', figsize=(30, 6), linewidth=5, color=['red', 'darkblue'])
    # Beschriftungen und Anpassungen
    plt.xlabel('Tag')
    plt.ylabel('Windgeschwindigkeit in m/s')
    plt.suptitle(Ortschaft, fontsize=14, fontweight='bold')
    plt.title(f'Wind Vergleich während Lothar und dem Jahr {Vergleichsjahr}')
    
    plt.legend(loc='upper left')
    plt.grid(False)
    # Plot-Begrenzungen oben und rechts entfernen
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.show()